In [1]:
import json
import os
import pandas as pd
import numpy as np
from flatten_dict import flatten
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import calendar
import warnings
from scipy.sparse import hstack,csr_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
import hdbscan
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
from bayes_opt import BayesianOptimization
from sklearn.metrics import silhouette_score
from sklearn.ensemble import IsolationForest
from itertools import combinations
from collections import Counter
import networkx as nx
import matplotlib.colors as mcolors
import ast
from networkx.algorithms import bipartite
from community import community_louvain
from pyvis.network import Network
from IPython.core.display import display, HTML
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
import category_encoders as ce
warnings.filterwarnings('ignore')

C:\Users\natha\AppData\Local\Temp\ipykernel_24076\1342185436.py:29: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


**extract_tweet_hashtag**

---


Cette fonction vise à extraire des informations détaillées liées au tweet lui-même, y compris les hashtags qu'il contient. Elle retourne un dictionnaire contenant l'identifiant du tweet, la date de création, la langue, le code pays et le nom du pays (si disponibles), le type et le nom du lieu (si disponibles), les coordonnées (si disponibles), le nombre de hashtags et une liste des hashtags présents dans le tweet.

**extract_quote_status**

---


Cette fonction extrait les informations concernant le statut cité dans un tweet (si le tweet est une citation d'un autre tweet). Elle renvoie un dictionnaire avec l'identifiant du tweet cité, l'identifiant utilisateur sous forme de chaîne de caractères et le nom d'utilisateur (screen name) de l'auteur du tweet cité.

**extract_user_mentions**

---


Cette fonction est conçue pour récupérer les informations relatives aux mentions d'utilisateurs dans un tweet. Elle produit un dictionnaire comprenant le nombre de mentions d'utilisateurs, une liste de leurs identifiants sous forme de chaîne de caractères, et une liste de leurs noms d'utilisateur.

**extract_hashtag_retweet**

---


Cette fonction se focalise sur l'extraction des hashtags présents dans un retweet. Elle retourne un dictionnaire contenant l'identifiant du tweet original retweeté et une liste des hashtags qui y sont associés.

**extract_user_info**

---


Finalement, cette fonction extrait les informations liées à l'utilisateur qui a posté le tweet. Le dictionnaire retourné inclut l'identifiant de l'utilisateur, le nombre d'amis (autres utilisateurs suivis par cet utilisateur) et la date de création du compte de l'utilisateur.

In [2]:
def extract_tweet_hashtag(tweet):
    hashtag_info = {
        'tweet_id': tweet.get('id'),
        'tweet_created_at': tweet.get('created_at'),
        'tweet_lang': tweet.get('lang'),
        'tweet_country_code': None,
        'tweet_country': None,
        'tweet_place_type': None,
        'tweet_place_name': None,
        'tweet_coordinates': [],
        'tweet_nbr_hashtags':None,
        'tweet_hashtags': []
    }

    place = tweet.get('place',None)
    if place:
        hashtag_info.update({
            'tweet_country_code': tweet.get('place.country_code',None),
            'tweet_country': tweet.get('place.country',None),
            'tweet_place_type': tweet.get('place.place_type',None),
            'tweet_place_name': tweet.get('place.name',None),
            'tweet_coordinates': tweet.get('place.bounding_box.coordinates',[])
        })

    hashtags = tweet.get('entities.hashtags',None)
    if hashtags:
        hashtag_info.update({
            'tweet_nbr_hashtags': len(hashtags),
            'tweet_hashtags': [hashtag['text'] for hashtag in hashtags if 'text' in hashtag]
        })

    return hashtag_info

def extract_quote_status(tweet):
    quote_status_info = {
        'quoted_status_id_tweet':None,
        'quote_status_id_str':None,
        'quote_status_id_screen_name':None,
    }

    is_quote_status = tweet.get('is_quote_status',False)
    if is_quote_status:
        quote_status_info.update({
            'quoted_status_id_tweet':tweet.get('quoted_status.id_str',None),
            'quote_status_id_str': tweet.get('quoted_status.user.id_str',None),
            'quote_status_id_screen_name': tweet.get('quoted_status.user.screen_name',None)
        })

    return quote_status_info

def extract_user_mentions(tweet):
    user_mentions_info = {
        'user_mentions_nbr':None,
        'user_mentions_ids_str': [],
        'user_mentions_screen_name':[]
    }

    user_mentions = tweet.get('entities.user_mentions',None)
    if user_mentions:
        user_mentions_info.update({
            'user_mentions_nbr': len(user_mentions),
            'user_mentions_ids_str' : [mention.get('id_str') for mention in user_mentions],
            'user_mentions_screen_name': [mention.get('screen_name') for mention in user_mentions]
        })

    return user_mentions_info

def extract_hashtag_retweet(tweet):
    retweet_info = {
        'retweet_id': tweet.get('retweeted_status.id_str',None),
        'retweet_hashtags':[]
    }

    hashtags_retweet = tweet.get('retweeted_status.entities.hashtags',None)
    if hashtags_retweet:
        retweet_info.update({
            'retweet_hashtags_nbr': len(hashtags_retweet),
            'retweet_hashtags': [hashtag['text'] for hashtag in hashtags_retweet if 'text' in hashtag]
        })

    return retweet_info

def extract_user_info(tweet):
    return {
        'user_id': tweet.get('user.id',None),
        'user_screen_name': tweet.get('user.screen_name',None),
        'user_friends_count': tweet.get('user.friends_count',None),
        'user_create_at': tweet.get('user.created_at',None)
    }

In [5]:
# %%time
def convertion_json(json_object):
    converted_json = json.loads(json_object)
    converted_json = flatten(converted_json, reducer='dot')
    return converted_json

def readjsonfiles3(directory_path):
    to_return = []
    files_in_dir = os.listdir(directory_path)

    for filename in files_in_dir:
        if filename.endswith('.json'):
            file_path = os.path.join(directory_path, filename)
            file = open(file_path, 'r',encoding='utf-8')
            lines = file.readlines()
            for line in lines:
                tweet = convertion_json(line)
                dict_extract_user_info = extract_user_info(tweet)
                dict_extract_tweet_hashtag = extract_tweet_hashtag(tweet)
                dict_extract_user_mentions = extract_user_mentions(tweet)
                dict_extract_hashtag_retweet = extract_hashtag_retweet(tweet)
                dict_extract_quote_status = extract_quote_status(tweet)

                dict_extract_concatene = {**dict_extract_user_info,
                            **dict_extract_tweet_hashtag,
                            **dict_extract_user_mentions,
                            **dict_extract_hashtag_retweet,
                            **dict_extract_quote_status}
                to_return.append(dict_extract_concatene)
            file.close()
    return to_return

files_directory = "../../../../raw"
tweet_list_enrichi = readjsonfiles3(files_directory)

# Interprétation des indicateurs en sortie de l'API

In [ ]:
# Exemple d'extraction d'indicateur pour un tweet avec les fonction ci-dessus
{'user_id': 39084553,
  'user_friends_count': 826,
  'user_create_at': 'Sun May 10 17:53:13 +0000 2009',
  'tweet_id': 1007083805889449985,
  'tweet_created_at': 'Thu Jun 14 02:14:24 +0000 2018',
  'tweet_lang': 'en',
  'tweet_country_code': None,
  'tweet_country': None,
  'tweet_place_type': None,
  'tweet_place_name': None,
  'tweet_coordinates': [],
  'tweet_nbr_hashtags': 2,
  'tweet_hashtags': ['WorldCup', 'United2026'],
  'user_mentions_nbr': 1,
  'user_mentions_ids_str': ['19077044'],
  'user_mentions_screen_name': ['HoustonRockets'],
  'retweet_id': '1006976904585498624',
  'retweet_hashtags': ['WorldCup', 'United2026'],
  'quoted_status_id_tweet': None,
  'quote_status_id_str': None,
  'quote_status_id_screen_name': None}

Identifiant de l'utilisateur (user_id): Le tweet a été publié par l'utilisateur ayant l'identifiant 39084553.

---


Nombre d'amis de l'utilisateur (user_friends_count): L'utilisateur avait 826 amis au moment de la publication du tweet.

---


Date de création du compte utilisateur (user_create_at): Le compte de l'utilisateur a été créé le 10 mai 2009 à 17h53 UTC.

---


Identifiant du tweet (tweet_id): L'identifiant unique de ce tweet est 1007083805889449985.

---


Date de création du tweet (tweet_created_at): Le tweet a été publié le 14 juin 2018 à 02h14 UTC.

---


Langue du tweet (tweet_lang): Le tweet est en anglais (en).

---


Code du pays et nom du pays (tweet_country_code, tweet_country): Ces informations ne sont pas disponibles pour ce tweet (None), ce qui signifie qu'il n'est pas associé à un pays spécifique.

---


Type et nom du lieu (tweet_place_type, tweet_place_name): Ces informations ne sont également pas disponibles (None), indiquant que le tweet n'a pas été publié avec des informations de localisation précises.

---


Coordonnées (tweet_coordinates): Aucune coordonnée géographique n'est associée à ce tweet, comme indiqué par une liste vide [].

---


Nombre de hashtags et hashtags utilisés (tweet_nbr_hashtags, tweet_hashtags): Le tweet contient 2 hashtags, WorldCup et United2026.

---


Nombre de mentions d'utilisateurs et informations sur ces mentions (user_mentions_nbr, user_mentions_ids_str, user_mentions_screen_name): Le tweet mentionne 1 utilisateur, avec l'identifiant 19077044 et le nom d'écran HoustonRockets.

---


Informations sur le retweet (retweet_id, retweet_hashtags): Ce tweet est un retweet de l'identifiant 1006976904585498624, et il contient les mêmes hashtags que le tweet original, à savoir WorldCup et United2026.

---


Statut de citation (quoted_status_id_tweet, quote_status_id_str, quote_status_id_screen_name): Ce tweet ne cite pas un autre tweet, comme indiqué par les valeurs None.

# Importation des données dans un dataframe

In [4]:
df_initial = pd.DataFrame(tweet_list_enrichi)
df_initial['tweet_created_at'] = pd.to_datetime(df_initial['tweet_created_at'], format='%a %b %d %H:%M:%S +0000 %Y')
# df_initial = pd.read_csv("generated_dataframe.csv", sep=",", header=0)
df_initial.set_index('tweet_created_at', inplace=True)
print(df_initial.shape)
df_initial.head()

NameError: name 'tweet_list_enrichi' is not defined

In [ ]:
df_initial["user_screen_name"] = df_initial["user_screen_name"].astype(str)

## Exploration

In [ ]:
df = df_initial.copy()
# Mettre en minuscule tous les hashtags et user mentions avant le filtrage
df['user_screen_name'] = df['user_screen_name'].apply(lambda x: x.lower())
df['tweet_hashtags'] = df['tweet_hashtags'].apply(lambda x: [tag.lower() for tag in x])
df['user_mentions_screen_name'] = df['user_mentions_screen_name'].apply(lambda x: [tag.lower() for tag in x])
df['retweet_hashtags'] = df['retweet_hashtags'].apply(lambda x: [tag.lower() for tag in x])

In [ ]:
all_hashtags_tweet = pd.Series([hashtag for sublist in df['tweet_hashtags'].dropna() for hashtag in sublist])
all_hashtags_tweet.reset_index(drop=True, inplace=True)
hashtag_tweet_frequence = all_hashtags_tweet.value_counts().sort_values(ascending = False)[:50] # top 50 des hashtags utilisés dans les tweets
print(hashtag_tweet_frequence)
hashtags_plot = list(hashtag_tweet_frequence.index)

In [ ]:
hashtag_tweet_frequence.plot(kind='bar')
plt.title('Top 50 des hashtags utilisés dans les tweets')
plt.xlabel('Hashtags tweet')
plt.ylabel('Fréquence')
plt.xticks(rotation=90, fontsize=7)
plt.show()

In [ ]:
all_hashtags_retweet = pd.Series([hashtag for sublist in df['retweet_hashtags'].dropna() for hashtag in sublist])
all_hashtags_retweet.reset_index(drop=True, inplace=True)
hashtag_retweet_frequence = all_hashtags_retweet.value_counts().sort_values(ascending = False)[:50] # top 50 des hashtags utilisés dans les retweets
print(hashtag_retweet_frequence)
hashtags_plot = list(hashtag_retweet_frequence.index)

In [ ]:
hashtag_retweet_frequence.plot(kind='bar')
plt.title('Top 50 des hashtags utilisés dans les retweets')
plt.xlabel('Hashtags retweet')
plt.ylabel('Fréquence')
plt.xticks(rotation=90, fontsize=7)
plt.show()

In [ ]:
all_user_tweet = pd.Series([user for user in df['user_screen_name'].dropna()])
all_user_tweet.reset_index(drop=True, inplace=True)
user_tweet_frequence = all_user_tweet.value_counts().sort_values(ascending = False)[:50] # top 50 des users qui publient le plus de tweet
print(user_tweet_frequence)
hashtags_plot = list(user_tweet_frequence.index)

In [ ]:
user_tweet_frequence.plot(kind='bar')
plt.title('Top 50 des users qui publient le plus de tweet')
plt.xlabel('users')
plt.ylabel('Fréquence')
plt.xticks(rotation=90, fontsize=7)
plt.show()

In [ ]:
users_plus_actif = df['user_screen_name'].value_counts().head(50).index # top 50 des users qui publient le plus de tweet
df_filtre_user_plus_actif = df[df['user_screen_name'].isin(users_plus_actif)]
df_nbr_moyen_hashtags_user = df_filtre_user_plus_actif.groupby('user_screen_name')['tweet_nbr_hashtags'].mean()
print(df_nbr_moyen_hashtags_user)

In [ ]:
df_nbr_moyen_hashtags_user.plot(kind='bar')
plt.title('Nombre de hashtags moyen dans les tweets des 50 users les plus actifs')
plt.xlabel('users')
plt.ylabel('Fréquence')
plt.xticks(rotation=90, fontsize=7)
plt.show()

## Préparation des données

In [ ]:
df_filtre = df.copy()
# Filtrer les lignes pour exclure les tweets contenant worldcup dans les hashtags de tweets ou de retweets
df_filtre = df_filtre[
    ~(
        df_filtre['tweet_hashtags'].apply(lambda x: any('worldcup' in tag.lower() for tag in x)) |
        df_filtre['retweet_hashtags'].apply(lambda x: any('worldcup' in tag.lower() for tag in x))
    )
]
print(df_filtre.shape)
df_filtre.head()

### Préparation des données

In [ ]:
df_preparation= df_filtre.copy()

# Transformation des listes en chaînes séparées par des espaces pour TF-IDF
vectorizer = TfidfVectorizer()

df_preparation['hashtags_str'] = df_preparation['tweet_hashtags'].apply(lambda x: ' '.join(x))
df_preparation['user_mentions_str'] = df_preparation['user_mentions_screen_name'].apply(lambda x: ' '.join(x))
df_preparation['retweet_hashtags_str'] = df_preparation['retweet_hashtags'].apply(lambda x: ' '.join(x))

X_hashtags = vectorizer.fit_transform(df_preparation['hashtags_str'])
X_user_mentions = vectorizer.fit_transform(df_preparation['user_mentions_str'])
X_retweet_hashtags = vectorizer.fit_transform(df_preparation['retweet_hashtags_str'])

# Encodage les variables temporelles
df_preparation['year_month'] = df_preparation.index.month
df_preparation['hour'] = df_preparation.index.hour
df_preparation['minute'] = df_preparation.index.minute
df_preparation['second'] = df_preparation.index.second

# Cyclisation des mois (12 mois dans une année)
df_preparation['month_sin'] = np.sin(2 * np.pi * df_preparation['year_month'] / 12)
df_preparation['month_cos'] = np.cos(2 * np.pi * df_preparation['year_month'] / 12)

# Cyclisation des heures (24 heures dans une journée)
df_preparation['hour_sin'] = np.sin(2 * np.pi * df_preparation['hour'] / 24)
df_preparation['hour_cos'] = np.cos(2 * np.pi * df_preparation['hour'] / 24)

# Cyclisation des minutes (60 minutes dans une heure)
df_preparation['minute_sin'] = np.sin(2 * np.pi * df_preparation['minute'] / 60)
df_preparation['minute_cos'] = np.cos(2 * np.pi * df_preparation['minute'] / 60)

# Cyclisation des secondes (60 secondes dans une minutes)
df_preparation['second_sin'] = np.sin(2 * np.pi * df_preparation['second'] / 60)
df_preparation['second_cos'] = np.cos(2 * np.pi * df_preparation['second'] / 60)

temporal_features = ['month_sin', 'month_cos', 'hour_sin', 'hour_cos', 'minute_sin', 'minute_cos', 'second_sin', 'second_cos']
X_temporal = csr_matrix(df_preparation[temporal_features].values)

# Combinaison des embeddings tf-idfet des encodages temporels
X_combined = hstack([X_hashtags, X_user_mentions, X_retweet_hashtags, X_temporal])
print(X_combined.shape)